In [1]:
import tensorflow as tf
import tf_encrypted as tfe
from tf_encrypted.keras.losses import BinaryCrossentropy
from tf_encrypted.keras.optimizers import SGD

from common import DataOwner

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/justin/anaconda3/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0.so'


ImportError: cannot import name 'DataOwner' from 'common' (/home/justin/anaconda3/lib/python3.7/site-packages/common/__init__.py)

In [ ]:
@tfe.local_computation('input-provider')
def provide_input():
    # normal TensorFlow operations can be run locally
    # as part of defining a private input, in this
    # case on the machine of the input provider
    return tf.ones(shape=(5, 10))

# define inputs
w = tfe.define_private_variable(tf.ones(shape=(10,10)))
x = provide_input()

# define computation
y = tfe.matmul(x, w)

with tfe.Session() as sess:
    # initialize variables
    sess.run(tfe.global_variables_initializer())
    # reveal result
    result = sess.run(y.reveal())

In [ ]:
x

In [ ]:
w

In [ ]:
result

In [ ]:
num_features = 10
batch_size = 100
steps_per_epoch = (training_set_size // batch_size)
epochs = 20

# Provide encrypted training data
data_owner = DataOwner('data-owner', batch_size)
x_train, y_train = data_owner.provide_private_training_data()

# Define model
model = tfe.keras.Sequential()
model.add(tfe.keras.layers.Dense(1, batch_input_shape=[batch_size, num_features]))
model.add(tfe.keras.layers.Activation('sigmoid'))

# Specify optimizer and loss
model.compile(optimizer=SGD(lr=0.01),
              loss=BinaryCrossentropy())

# Start training
model.fit(x_train,
          y_train,
          epochs=epochs,
          steps_per_epoch=steps_per_epoch)